### Agentic RAG

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool
from langchain_community.utilities import GoogleSerperAPIWrapper

In [ ]:
import os

from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

In [ ]:
load_dotenv()

azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_deployment = os.getenv("AZURE_DEPLOYMENT_NAME")
azure_api_version = os.getenv("AZURE_API_VERSION")

llm = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    azure_deployment=azure_deployment,
    api_version=azure_api_version,
    max_tokens=2000,
    temperature=0.1
)

In [ ]:
@tool("GoogleSearch")
def search(query_string: str):
    """
    Useful to search for any kinds of information and
    when you need to search the internet for any kinds of information, use this tool.
    Prefer this tool when you search for long queries.
    Should not be used for Article search or Topic Search.
    """
    
    search = GoogleSerperAPIWrapper()
    
    return search.run(query_string)

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
wiki = WikipediaQueryRun(
    name="WikiepdiaSearch",
    description="Use this tool when you want to analyze for information on Wikipedia by Terms, Keywords or any Topics.",
    api_wrapper=api_wrapper)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
azure_embedding_deployment = os.getenv(
    "AZURE_EMBEDDING_DEPLOYMENT_NAME", "text-embedding-3-large")

loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
).split_documents(docs)

embeddings = AzureOpenAIEmbeddings(azure_endpoint=azure_endpoint,
                                    model="text-embedding-3-large",
                                    api_key=azure_api_key,
                                    deployment=azure_embedding_deployment,
                                    dimensions=3072,
                                    )
vectordatabase = FAISS.from_documents(documents, embeddings)
retriever = vectordatabase.as_retriever()

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "search for information about langsmith. for any questions related to langsmith, you must use this tool"
)

retriever_tool.name

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

In [ ]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [ ]:
tools = [arxiv, search, wiki, retriever_tool]
tools

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, SystemMessage

agent_executor = create_react_agent(
    llm,
    tools = tools
)

In [ ]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="tell me about langsmith")
    ]
}):
    print(stream)
    print("***********")

In [ ]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="whats the paper 2412.16446 talk about it?")
    ]
}):
    print(stream)
    print("***********")

In [ ]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Indian Constitution")
    ]
}):
    print(stream)
    print("***********")

In [ ]:
for stream in agent_executor.stream({
    "messages": [
        HumanMessage(content="Who won the General Elections in India in 2024?")
    ]
}):
    print(stream)
    print("***********")